# Gradiente descendente Estocástico

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Importando .csv
df = pd.read_csv('dataset/train_data.csv', header=None,  skiprows=[0], usecols=[0,1,2])

In [ ]:
# Função FunkSVD
def funkSVD(R, k, lr = 0.05, reg = 0.02, miter = 10):
    global_mean = R[2].mean(skipna = True)
    nusers = df[0].max()
    nitems = df[1].max()
    bu = np.full(nusers+1, 0)
    bi = np.full(nusers+1, 0)
    P = np.full((nusers+1, k), 0.1)
    Q = np.full((nitems+1, k), 0.1)
    for f in range(k):
        for l in range(0, miter):
            for j in range(0, len(R)):
                u = R[0][j]
                i = R[1][j]
                r_ui = R[2][j]
                pred = global_mean + bu[u] + bi[i] + np.dot(P[u, ], Q[i, ])
                e_ui = r_ui - pred
                bu[u] = bu[u] + lr * e_ui # - reg * bu[u]
                bi[i] = bi[i] + lr * e_ui # - reg * bi[i]
                temp_uf = P[u, f]
                P[u, f] = P[u, f] + lr * (e_ui * Q[i, f] - reg * P[u, f])
                Q[i, f] = Q[i, f] + lr * (e_ui * temp_uf - reg * Q[i, f])

    return { "global_mean": global_mean, "bu": bu, "bi": bi, "P": P, "Q": Q }

In [ ]:
funk = funkSVD(df, 4)

In [ ]:
# nota do usuário id. 500 para o item id. 100:
# pred <- funk$global_mean + funk$bu[500]	+ funk$bi[100] + funk$P[500,] %*% funk$Q[100,]	

In [3]:
# Função Otimizado
def svdopt(R, k, lr = 0.05, reg = 0.02,miter = 10):
    global_mean = R[2].mean(skipna = True)
    nusers = df[0].max()+1
    nitems = df[1].max()+1
    bu = np.full(nusers, 0, dtype=float)
    bi = np.full(nusers, 0, dtype=float)
    P = np.random.normal(loc = 0, scale = 0.1, size=(nusers, k))
    Q = np.random.normal(loc = 0, scale = 0.1, size=(nitems, k))
    error = list()
    for l in range(0, miter):
        sq_error = 0
        for j in range(0, len(R)):
            u = R[0][j]
            i = R[1][j]
            r_ui = R[2][j]
            pred = global_mean + bu[u] + bi[i] + np.dot(P[u, ], Q[i, ])
            e_ui = r_ui - pred
            sq_error += e_ui**2
            bu[u] += lr * e_ui # - reg * bu[u]
            bi[i] += lr * e_ui # - reg * bi[i]
            for f in range(k):
                temp_uf = P[u, f]
                P[u, f] = P[u, f] + lr * (e_ui * Q[i, f] - reg * P[u, f])
                Q[i, f] = Q[i, f] + lr * (e_ui * temp_uf - reg * Q[i, f])
        error.append(math.sqrt(sq_error/len(R)))

    return { "global_mean": global_mean, "bu": bu, "bi": bi, "P": P, "Q": Q, "error": error }

In [4]:
svdopt = svdopt(df, 4)

In [5]:
# Predizendo nota
pred = svdopt["global_mean"] + svdopt["bu"][300] + svdopt["bi"][683] + np.dot(svdopt["P"][300], svdopt["Q"][100])
pred

4.6138285188334525